<a href="https://colab.research.google.com/github/yojulab/study_AIs/blob/main/docs/NLPs/NLP_classification_ynanewstitles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 준비
- DDA, EDA 확인

In [ ]:
import pandas as pd
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
topic_dict = pd.read_csv('topic_dict.csv')

In [ ]:
# train[:2]
# test[:2]
topic_dict

,topic,topic_idx
0,IT과학,0
1,경제,1
2,사회,2
3,생활문화,3
4,세계,4
5,스포츠,5
6,정치,6


### 데이터 전처리
- trainset, testset 병합
- tokenizing : 머신 맞게 vector

In [ ]:
raw_concat = pd.concat([train, test])
train.shape, test.shape, raw_concat.shape

((45654, 3), (9131, 2), (54785, 3))

In [ ]:
raw_concat.columns

Index(['index', 'title', 'topic_idx'], dtype='object')

In [ ]:
# 숫자 삭제
raw_concat['title_preprocess'] = raw_concat['title'].str.replace('[0-9]','',regex=True)

In [ ]:
# 영문자 소문자로 통일
raw_concat['title_preprocess'] = raw_concat['title_preprocess'].str.lower()

In [ ]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 42.0 MB/s eta 0:00:00


In [ ]:
from konlpy.tag import Okt
okt = Okt()

In [ ]:
# 조사, 어미, 구두점 같은 것을 제거
def okt_clean(text):
    clean_text = []
    for word in okt.pos(text, stem=True):
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']:
            clean_text.append(word[0])

    return " ".join(clean_text)

In [ ]:
raw_concat['title_okt_clean'] = raw_concat['title_preprocess'].apply(okt_clean)

In [ ]:
raw_concat[:5]

,index,title,topic_idx,title_preprocess,title_okt_clean
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4.0,인천→핀란드 항공기 결항…휴가철 여행객 분통,인천 → 핀란드 항공기 결항 휴가 철 여행객 분통
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4.0,실리콘밸리 넘어서겠다…구글 조원 들여 美전역 거점화,실리콘밸리 넘어서다 구글 조원 들이다 美 전역 거점 화
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4.0,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,이란 외무 긴장 완화 해결 책 미국 경제 전쟁 멈추다 것
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4.0,nyt 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,nyt 클린턴 측근 韓 기업 특수 관계 조명 공과 사 맞다 물리다 종합
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4.0,시진핑 트럼프에 중미 무역협상 조속 타결 희망,시진핑 트럼프 중미 무역 협상 조속 타결 희망


In [ ]:
# tokenizing
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(tokenizer=None)
# TfidfVectorizer(tokenizer=None, ngram_range=(1,2))

In [ ]:
tfidfVectorizer.fit(raw_concat['title_okt_clean'])

TfidfVectorizer()

In [ ]:
# trainset, testset 분리
trainset = raw_concat[raw_concat['topic_idx'].notnull()]   # train
testset = raw_concat[raw_concat['topic_idx'].isnull()]   # test
trainset.shape, testset.shape

((45654, 5), (9131, 5))

## 모델

In [ ]:
train_target = trainset['topic_idx']
train_feature = tfidfVectorizer.transform(trainset['title_okt_clean'])  # 머신 학습 위한 선 작업

In [ ]:
from sklearn.ensemble import RandomForestClassifier
randomForestClassifier = RandomForestClassifier(n_jobs=-1)

In [ ]:
randomForestClassifier.fit(train_feature, train_target)

RandomForestClassifier(n_jobs=-1)

## 평가